<div class="alert alert-block alert-success">
    <h1 align="center">Information Retrival Systems</h1>
    <h2 align="center">Simple search engine project</h2>
    <h3 align="center">Phase 2</h3>
</div>

## Loading Libraries

In [1]:
from parsivar import Normalizer, Tokenizer, FindStems
from hazm import stopwords_list
import pandas as pd
import pickle
import math
import re

## Loading raw and processed Data
- loading the json file as a dataframe and also transpose it
- loading preprocessed content
- loading inverted index
- create dictionary according to inverted index

In [2]:
df = pd.read_json('IR_data_news_12k.json')
df = df.transpose()

In [3]:
def load_file(filename):
    my_file = None
    with open(filename, 'rb') as inp:
        my_file = pickle.load(inp)
    return my_file

In [4]:
preprocessed_content = load_file('./Processed data/preprocessed_content.pkl')

In [5]:
inverted_index = load_file('./Processed data/inverted_index.pkl')

In [6]:
dictionary = list(inverted_index.keys())

# Step1)
## Modeling documents in vector space
- Here we should define a function that calculate tf_idf score. The formula explained below:
$$tfidf(t, d, D) = tf(t, d) \times idf(t, D) = \ (1 + log(f_{t, d})) \times \log(\frac{N}{n_t})$$    

In [7]:
def tfidf(term, doc, doc_collection, total_number_of_docs):
    # idf is a constant value for each token and it doesnt depend on query
    N = total_number_of_docs
    nt = doc_collection[term]['doc_frequency']
    idf = math.log10(N / nt)
    
    # now we should compute number of term in that doc
    term_count = doc.count(term)
    if term_count == 0:
        tf = 0
    else:
        tf = 1+math.log10(term_count)
    
    return  tf*idf

In [8]:
print(tfidf('خبرگزاری',df.iloc[3]['content'], inverted_index , len(df)))

0.007249774608743844


# Step2)
## Answering the queries in the vector space  

- Here we should define a function for calculating cosine similarity. Also we define a function to compute Doc_length.
- For this part we help from pseudo code in slides.


In [9]:
def get_length(doc_id):
    l = 0
    for token in preprocessed_content[doc_id]:
        l += tfidf(term=token, doc=preprocessed_content[doc_id], doc_collection=inverted_index, total_number_of_docs=len(df))**2
    return l

In [10]:
def cosine(query, k, inverted_index, is_champion=True):
    docID_score = {}
    doc_ids = []
    for term in query:
        # step 1: calculate weight of term in the query
        weight_in_query = tfidf(term=term, doc=query, doc_collection=inverted_index, total_number_of_docs=len(df))
        # step 2: get postings list for term (if is_champion is true we should use ch)
        if is_champion:
            postings_list = inverted_index[term]['champion']
        else:
            postings_list = inverted_index[term]['posting_list']
            
        for posting in postings_list:
            if is_champion:
                doc_id = posting[0]
            else:
                doc_id = list(posting.keys())[0]
            # step 3: calculate weight of term in the Doc
            weight_in_doc = tfidf(term=term, doc=df.iloc[doc_id]['content'],  doc_collection=inverted_index, total_number_of_docs=len(df))
            
            # step4:save or update related score of a doc
            if docID_score.get(doc_id):
                docID_score[doc_id]['score'] += weight_in_query * weight_in_doc
                #docID_score[doc_id]['squar length'] += (weight_in_doc ** 2)
            else:
                doc_ids.append(doc_id)
                #docID_score[doc_id]= {'squar length': weight_in_doc ** 2, 'score':weight_in_query * weight_in_doc}
                docID_score[doc_id]= {'squar length': get_length(doc_id), 'score':weight_in_query * weight_in_doc}
    # step5: normalizing
    new_docID_score = []
    for doc_id in doc_ids:
        docID_score[doc_id]['score'] = docID_score[doc_id]['score']/math.sqrt(docID_score[doc_id]['squar length'])
        new_docID_score.append((doc_id,docID_score[doc_id]['score']))
    # step6: select k best:
    new_docID_score = [(docID_score[0],docID_score[1]['score']) for docID_score in docID_score.items()]
    if k > len(new_docID_score):
        k = len(new_docID_score)
    k_best = []
    for i in range(k):
        max_score = max(new_docID_score,key=lambda x:x[1])
        k_best.append(max_score)
        new_docID_score.remove(max_score)
    return k_best

# Step3)
## Increasing the speed of processing of queries
- In this section we should define a function to copy prevoius inverted index and add **champion lists** to our new inverted index.

In [11]:
def create_champions(r):
    index_with_champion = inverted_index
    for token in index_with_champion:
        new_docID_tf = []
        for posting in index_with_champion[token]['posting_list']:
            new_docID_tf.append((list(posting.keys())[0], posting[list(posting.keys())[0]]['term_frequency']))
            
        inverted_index[token]['champion'] = []
        
        if r>len(inverted_index[token]['posting_list']):
            r= len(inverted_index[token]['posting_list'])
            
        for i in range(r):
            max_tf = max(new_docID_tf,key=lambda x:x[1])
            inverted_index[token]['champion'].append(max_tf)
            new_docID_tf.remove(max_tf)    
    return index_with_champion

- So we can call above function to create champion lists. each list just has r postings or less. I consider r equal to 6

In [12]:
index_with_champion = create_champions(r = 6)

In [13]:
index_with_champion['گزارش']['champion']

[(11604, 31), (11449, 30), (9500, 29), (11163, 28), (11774, 26), (11686, 25)]

- now we can define a function to show results

In [14]:
def show_result(results):
    if len(results)==0:
        print("There isn't any doc related to this query")
    else:
        for result in results:
            print(50* "/\\")
            print(f'docID: {result[0]}')
            print(f'Score: {result[1]}')
            print(f'Title: {df.loc[result[0]]["title"]}')
            print(f'URL: {df.loc[result[0]]["url"]}')
            print(f'{df.loc[result[0]]["content"]}')
            print(50*"/\\")               

- We should define total preprocess function and some other functions again. its the same as phase1

In [15]:
def remove_punc (input_content):
    return re.sub(r'[^\w\s]','',input_content)

def stemmer (tokens):
    stemmed = []
    st = FindStems()
    for i in tokens:
        stemmed_token = st.convert_to_stem(i)
        #stemmed_token = stemmed_token.split('&')
        stemmed.append(stemmed_token)
    return stemmed

def stopwords_remover(stemmed):
    stop_words = stopwords_list()
    
    for i in stemmed:
        if i in stop_words:
            stemmed.remove(i)
    return stemmed

def total_preprocess (input_content, is_stopwords_remove=True, is_stemming=True):
  
    # 1) first we remove punctuations
    punc_removed = remove_punc(input_content)

    # 2) second we Normalize it
    normal_content = Normalizer().normalize(punc_removed)
      
    # 3) then we can tokenize content
    updated_content = Tokenizer().tokenize_words(normal_content)
      
    # 4) then we remove stopwords
    # (Attention: in the parsivar library there isn't any functions to remove stopwords. so we use hazm for this part)
    if is_stopwords_remove:
        updated_content = stopwords_remover(updated_content)
      
    # 5) then we stemming
    if is_stemming:
        updated_content = stemmer(updated_content)
      
    return updated_content

- also we need an interface that get user query and then show related results to him. I cosider k equal to 5. it means it only returns the first k that have the most score among the champion list or in posting list

In [16]:
def search(query, is_champion):
    preprocessed_query = total_preprocess(query)
    new_query = []
    for term in preprocessed_query:
        if term in dictionary:
            new_query.append(term)
    k_best = []
    if is_champion:
        k_best = cosine(query=new_query, k=5, inverted_index=index_with_champion, is_champion=is_champion)
    else:
        k_best = cosine(query=new_query, k=5, inverted_index=inverted_index, is_champion=is_champion)
        
    show_result(k_best)

# Step4)
## Report

Answer to queries in below modes:

- [ ] A query of simple and common single words

In [17]:
search('آسیا', is_champion=True)

/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\
docID: 1308
Score: 0.08388026952597244
Title: مصاحبه فارس با کارشناس فوتبال آسیا: لیگ بیست و یکم بدون سهمیه آسیایی نیست/فدراسیون تعیین کننده شد
URL: https://www.farsnews.ir/news/14001207000166/مصاحبه-فارس-با-کارشناس-فوتبال-آسیا-لیگ-بیست-و-یکم-بدون-سهمیه-آسیایی

به گزارش خبرنگار ورزشی خبرگزاری فارس، کنفدراسیون فوتبال آسیا روز گذشته اعلام کرد زمان بندی لیگ قهرمانان آسیا از فصل آینده به سال 2023 موکول می شود چون زمان برگزاری لیگ قهرمانان آسیا مثل فرمت اروپا از پاییز شروع می شود. این بدان معناست که بعد از برگزاری لیگ قهرمانان آسیا 2022 فصل آینده یعنی فصل 2022-2023 لیگ قهرمانان نیست و آغاز فصل جدید به فصل 2023-2024 موکول می شود. این تغییر در زمان بندی آغاز لیگ قهرمانان آسیا که از فصل بهار به پاییز موکول شد باعث می شود تا این شائبه به وجود آید که  این فصل از لیگ های غرب آسیا بدون سهمیه آسیایی باشند اما  AFC راهکار دیگری را برای این خلا ایجاد شده گذاشته است. در همین رابطه خبرنگار فارس به سراغ 

- [ ] A query of simple and common multi-word expressions

In [18]:
search('ورزشگاه آزادی', is_champion=True)

/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\
docID: 7819
Score: 0.06965646732492162
Title: فیلم| مسیرهای راهپیمایی خودرویی و موتوری22 بهمن در تهران
URL: https://www.farsnews.ir/news/14001120000538/فیلم|-مسیرهای-راهپیمایی-خودرویی-و-موتوری22-بهمن-در-تهران

به گزارش خبرگزاری فارس، مراسم راهپیمایی ۲۲ بهمن امسال در حدود ۱۵۰۰ شهر و سه هزار روستا به صورت همزمان در ساعت ۹ و ۳۰ دقیقه صبح آغاز خواهد شد.  مراسم راهپیمایی ۲۲ بهمن در تهران از مسیر‌های ۱۲ گانه با تفکیک مسیرخودرویی و موتوری برگزار خواهد شد و هموطنان می‌توانند با توجه به مسیر‌های اعلام شده در این مراسم شرکت کنند. براین اساس، مسیرهای راهپیمایی خودرویی و موتوری 22 بهمن به این شرح است:  مسیر شماره ۱ (موتوری):  مسجد امام حسین(ع) ، میدان امام حسین(ع) ، خیابان انقلاب ، خیابان آزادی ، میدان آزادی  مسیر شماره ۲ (خودرویی): مسجد جامع ابوذر ، میدان ابوذر ، خیابان ابوذر ، خیابان شهید آیت الله سعیدی ، میدان آزادی مسیر شماره ۳ (خودرویی):  مسجد امام هادی(ع) ، خیابان شهید آقایی ، میدان معلم ، خی

- [ ] A difficult and rarely repeated one-word query

In [19]:
search('استمهال', is_champion=True)

/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\
docID: 11229
Score: 0.7015721579670751
Title: وام کشاورزان استمهال شود
URL: https://www.farsnews.ir/news/14000819000217/وام-کشاورزان-استمهال-شود

به گزارش خبرنگار پارلمانی خبرگزاری فارس، محمد وحیدی نماینده بجنورد در مجلس شورای اسلامی در جلسه علنی امروز( چهارشنبه) در تذکری اصلاح قانون مجازات اسلامی درخصوص جرایم و قمار سایبری را اتفاق بسیار خوبی دانست و گفت: در حال حاضر بانک های خصوصی و دولتی برابر قانون در سال های خشکسالی موظف به استمهال وام های کشاورزان و دامداران و همچنین بخشودگی جرایم هستند اما هم اکنون طبق قانون اساسی عملکرد بانک ها در این زمینه ربا است. وی ادامه داد: مجلس باید به دولت تذکر دهد که تمام شعبات بانک ها به این قانون عمل کنند. مردم در شرایط فعلی با مشکلات زیادی مواجه هستند و امکان پرداخت وام های خود را ندارند. با وجودی که قانون مجوز داده که وام کشاورزان استمهال شود. بانک مرکزی باید دستورالعمل هایی که صادر می کند را پیگیری کند تا به درستی اجرا شود. در ادامه محمدباقر قالیبا

In [20]:
search('استمهال', is_champion=False)

/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\
docID: 11229
Score: 0.7015721579670751
Title: وام کشاورزان استمهال شود
URL: https://www.farsnews.ir/news/14000819000217/وام-کشاورزان-استمهال-شود

به گزارش خبرنگار پارلمانی خبرگزاری فارس، محمد وحیدی نماینده بجنورد در مجلس شورای اسلامی در جلسه علنی امروز( چهارشنبه) در تذکری اصلاح قانون مجازات اسلامی درخصوص جرایم و قمار سایبری را اتفاق بسیار خوبی دانست و گفت: در حال حاضر بانک های خصوصی و دولتی برابر قانون در سال های خشکسالی موظف به استمهال وام های کشاورزان و دامداران و همچنین بخشودگی جرایم هستند اما هم اکنون طبق قانون اساسی عملکرد بانک ها در این زمینه ربا است. وی ادامه داد: مجلس باید به دولت تذکر دهد که تمام شعبات بانک ها به این قانون عمل کنند. مردم در شرایط فعلی با مشکلات زیادی مواجه هستند و امکان پرداخت وام های خود را ندارند. با وجودی که قانون مجوز داده که وام کشاورزان استمهال شود. بانک مرکزی باید دستورالعمل هایی که صادر می کند را پیگیری کند تا به درستی اجرا شود. در ادامه محمدباقر قالیبا

- [ ] A difficult and rarely repeated multi-word query

In [21]:
search('ژئواستراتژیک و ژئوپلتیک ایران', is_champion=True)

/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\
docID: 8609
Score: 0.3766226478400331
Title: رئیسی: هیچ محدودیتی برای گسترش روابط با روسیه نداریم/ پوتین: دستورکار وسیعی پیش روی دو کشور است
URL: https://www.farsnews.ir/news/14001029000790/رئیسی-هیچ-محدودیتی-برای-گسترش-روابط-با-روسیه-نداریم-پوتین-دستورکار

به گزارش خبرگزاری فارس، آیت الله سید ابراهیم رئیسی رئیس‌جمهوری اسلامی ایران امروز چهارشنبه در دیدار با ولادیمیر پوتین رئیس جمهور روسیه، با اشاره به نقش تاثیرگذار دو قدرت ایران و روسیه در منطقه و جهان گفت: هیچ محدودیتی برای توسعه روابط با روسیه نداریم و روابط ممتاز تهران-مسکو در تراز مناسبات راهبردی است و ارتقا نیز خواهد یافت. روابط ایران و روسیه در مسیر مناسبات راهبردی قرار دارد آیت‌الله رئیسی ادراک مشترک دو کشور در مسائل منطقه‌ای و بین‌المللی را زمینه‌ساز همکاری‌های مشترک خواند و گفت: روابط ایران و روسیه در مسیر مناسبات راهبردی قرار دارد. تجربه موفق همکاری علیه تروریسم در سوریه، می‌تواند در قفقاز و افغانستان به کار گرفته شود رئیسی ت